In [180]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from math import pi

from bokeh.io import show, output_file
from bokeh.layouts import row
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource

# Read csvs

These are the csv's we cleaned up and combined in step 01. 

In [189]:
ings = pd.read_csv('data/ings.csv', encoding = "ISO-8859-1", index_col=0)
#prods = prods.drop_duplicates(subset = 'product')
prods = pd.read_csv('data/prods.csv', encoding = "ISO-8859-1", index_col=0)
#prods = prods.drop_duplicates(subset = 'product')
prod_ing = pd.read_csv('data/prod_ing.csv', encoding = "ISO-8859-1", index_col=0)
prod_ing = prod_ing.drop(['diff'], axis = 1)

print('Number of products: ', len(prods))
print('Number of unique ingredients: ', len(ings))
print('Number of total ingredients: ', len(prod_ing))

Number of products:  15945
Number of unique ingredients:  7325
Number of total ingredients:  545617


In [192]:
prod_ing.sort_values(['id', 'order'])

,id,ingredient,order
0,0,Water,1
1,0,Hydrogenated Polyisobutene,2
2,0,Butylene Glycol,3
3,0,Sesamum Indicum (Sesame) Seed Oil,4
4,0,Ammonium Acryloyldimethyltaurate/VP Copolymer,5
5,0,Sucrose,6
6,0,Hypnea Musciformis Extract,7
7,0,Gellidiela Acerosa Extract,8
8,0,Sialyllactose,9
9,0,Arginine,10


### Ings
Ings df is just a database of unique ingredients and their harmful score, function and notes.

In [182]:
#caught a rogue ingredient
ings['ingredient'] = ings['ingredient'].replace('Stearic Acid(Masking, Fragrance, Emulsion Stabilising, Emulsifying, Sufactant, Refatting, Surfactantsurfactant-Cleansing Agent Is Included As A Function For The Soap Form Of Stearic Acid.', 'Stearic Acid')
ings = ings.drop_duplicates(subset = 'ingredient')
print('Number of unique ingredients: ', len(ings))
ings.head()

Number of unique ingredients:  7324


,id,ingredient,ewg,cir,func_Abrasive,func_Antimicrobial,func_Antioxidant,func_Antistatic Agent,func_Astringent,func_Binding,...,notes_Comedogenic Rating (3),notes_Comedogenic Rating (4),notes_Comedogenic Rating (5),notes_Good for Dry Skin,notes_Good for Oily Skin,notes_Good for Sensitive Skin,notes_Paraben,notes_Promotes Wound Healing,notes_Sulfate,notes_UV Protection
0,0,Water,1,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,Hydrogenated Polyisobutene,1,A,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,Butylene Glycol,1,A,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,Sesamum Indicum (Sesame) Seed Oil,1,A,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,Ammonium Acryloyldimethyltaurate/VP Copolymer,1,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [183]:
ing_uniqueID = ings.loc[:,['ingredient']].reset_index()
ing_uniqueID = ing_uniqueID.drop(['index'], axis = 1)
ing_uniqueID['uniqueID'] = ing_uniqueID.index
ing_uniqueID

,ingredient,uniqueID
0,Water,0
1,Hydrogenated Polyisobutene,1
2,Butylene Glycol,2
3,Sesamum Indicum (Sesame) Seed Oil,3
4,Ammonium Acryloyldimethyltaurate/VP Copolymer,4
5,Sucrose,5
6,Hypnea Musciformis Extract,6
7,Gellidiela Acerosa Extract,7
8,Sialyllactose,8
9,Arginine,9


### Prods
Prods df is a database of all the products with various information including brand, price and rating

In [193]:
prods.head(3)

,id,product,brand,price,rating,ratingsCount,cat_Acne&BlemishTreatments,cat_Ampoules,cat_BodySkincare,cat_Cleansers,...,note_AlcoholFree,note_Anti-Aging,note_Brightening,note_ContainsAlcohol,note_ContainsParaben,note_ContainsSulfate,note_ParabenFree,note_PromotesWoundHealing,note_SulfateFree,note_UVProtection
0,0,Naturally Gentle Eye Makeup Remover,Clinique,20.00,5.0,16,0,0,0,0,...,1,0,0,0,1,0,0,0,1,0
2,2,Foot Cream Norwegian Formula,Neutrogena,23.82,4.9,84,0,0,0,0,...,1,0,1,0,1,0,0,1,1,0
3,3,Essential Power Skin Refiner_Moisture,Laneige,28.20,5.0,12,0,0,0,0,...,0,0,1,1,0,0,1,1,1,0


### Prod_ing
Prod ing is a database that essentiall links prods and ings. it contains the product id, ingredient and ingredient order.

In [187]:
prod_ing

,id,ingredient,order,uniqueID
0,0,Water,1,0
1,2,Water,1,0
2,3,Water,1,0
3,5,Water,1,0
4,7,Water,2,0
5,8,Water,1,0
6,9,Water,1,0
7,10,Water,1,0
8,11,Water,1,0
9,12,Water,1,0


### Add a column that is a unique ingredient identifier

In [186]:
#id is product id
#uniqueID is ingredient ID
prod_ing = pd.merge(prod_ing, ing_uniqueID, on='ingredient')
prod_ing.head()

,id,ingredient,order,uniqueID
0,0,Water,1,0
1,2,Water,1,0
2,3,Water,1,0
3,5,Water,1,0
4,7,Water,2,0


## Reformatting Prod_ing
Instead of having each ingredient in it's own row, we want to create a list of ingredients for each product while retaining the order.

In [167]:
prod_ing#.groupby('id')['ingredient'].apply(list)

,id,ingredient,order,uniqueID
0,0,Water,1,0
1,2,Water,1,0
2,3,Water,1,0
3,5,Water,1,0
4,7,Water,2,0
5,8,Water,1,0
6,9,Water,1,0
7,10,Water,1,0
8,11,Water,1,0
9,12,Water,1,0


In [159]:
#cleanup of a rogue ingredient here as well
prod_ing['ingredient'] = prod_ing['ingredient'].replace('Stearic Acid(Masking, Fragrance, Emulsion Stabilising, Emulsifying, Sufactant, Refatting, Surfactantsurfactant-Cleansing Agent Is Included As A Function For The Soap Form Of Stearic Acid.', 'Stearic Acid')

##group and create list
prod_ing_lists = prod_ing.groupby('id')['ingredient'].apply(list)
prod_ing_ID_lists = prod_ing.groupby('id')['uniqueID'].apply(list)

##convert back to dataframe and reset index
prod_ing_df = prod_ing_lists.to_frame().reset_index()
prod_ing_ID_df = prod_ing_ID_lists.to_frame().reset_index()
##check to make sure unique id's are still in tact (id shouldn't be exactly == to index)
prod_ing_lists.head()

KeyError: 'Column not found: uniqueID'

In [59]:
prod_ing_df.tail()

,id,ingredient
19858,19922,"[Water, Hydrogenated Polyisobutene, Disodium E..."
19859,19923,"[Water, Butylene Glycol, Sucrose, Disodium EDT..."
19860,19924,"[Water, Butylene Glycol, Glycerin, Allantoin, ..."
19861,19925,"[Tocopherol (Vitamin E, Tocopheryl Acetate, Be..."
19862,19926,"[Water, Butylene Glycol, Sucrose, Disodium EDT..."


In [65]:
prod_ing_lists = pd.merge(prod_ing_df, prod_ing_ID_df, on='id')
prod_ing_lists = prod_ing_lists.rename(columns = {'ingredient': 'ingList', 'uniqueID': 'ing#List' })
prod_ing_lists.head()

,id,ingList,ing#List
0,0,"[Water, Hydrogenated Polyisobutene, Butylene G...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
1,1,"[Disodium EDTA, Phenoxyethanol, Mineral Oil, G...","[15, 16, 22, 23, 24, 25, 26, 27, 28, 29, 30, 3..."
2,2,"[Water, Sodium Chloride, Methylparaben, Propyl...","[0, 13, 17, 19, 23, 46, 47, 48, 49, 50, 51, 52..."
3,3,"[Water, Butylene Glycol, Sucrose, Arginine, Di...","[0, 2, 5, 9, 15, 16, 23, 49, 62, 63, 64, 65, 6..."
4,4,"[Butylene Glycol, Disodium EDTA, Phenoxyethano...","[2, 15, 16, 17, 19, 23, 33, 47, 49, 92, 93, 94..."


## Merge prod_ings and prods
We can now use the list to merge back with the products.

In [97]:
products_and_ingredients= pd.merge(prod_ing_lists, prods, on='id')
products_and_ingredients.tail(3)

,id,ingList,ing#List,product,brand,price,rating,ratingsCount,cat_Acne&BlemishTreatments,cat_Ampoules,...,note_AlcoholFree,note_Anti-Aging,note_Brightening,note_ContainsAlcohol,note_ContainsParaben,note_ContainsSulfate,note_ParabenFree,note_PromotesWoundHealing,note_SulfateFree,note_UVProtection
16435,19923,"[Water, Butylene Glycol, Sucrose, Disodium EDT...","[0, 2, 5, 15, 16, 23, 26, 27, 45, 46, 49, 50, ...",Bright Lover Rubber Mask,Dr. Jart+,12.51,2.3,4,0,0,...,1,1,1,0,0,0,1,1,1,1
16436,19925,"[Tocopherol (Vitamin E, Tocopheryl Acetate, Be...","[45, 98, 100, 102, 119, 120, 135, 136, 137, 13...",Born Lippy Balms,The Body Shop,7.00,2.0,3,0,0,...,1,1,1,0,0,0,1,1,1,1
16437,19926,"[Water, Butylene Glycol, Sucrose, Disodium EDT...","[0, 2, 5, 15, 16, 23, 41, 50, 73, 85, 87, 109,...",Fresh Pressed Daily Booster With Pure Vitamin ...,Clinique,22.99,2.0,3,0,0,...,1,1,1,0,0,0,1,0,1,0


### Add column for number of ingredients

In [98]:
products_and_ingredients['ingCount'] = products_and_ingredients['ingList'].apply(lambda x: len(x))
products_and_ingredients.head(3)

,id,ingList,ing#List,product,brand,price,rating,ratingsCount,cat_Acne&BlemishTreatments,cat_Ampoules,...,note_Anti-Aging,note_Brightening,note_ContainsAlcohol,note_ContainsParaben,note_ContainsSulfate,note_ParabenFree,note_PromotesWoundHealing,note_SulfateFree,note_UVProtection,ingCount
0,0,"[Water, Hydrogenated Polyisobutene, Butylene G...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",Naturally Gentle Eye Makeup Remover,Clinique,20.00,5.0,16,0,0,...,0,0,0,1,0,0,0,1,0,22
1,2,"[Water, Sodium Chloride, Methylparaben, Propyl...","[0, 13, 17, 19, 23, 46, 47, 48, 49, 50, 51, 52...",Foot Cream Norwegian Formula,Neutrogena,23.82,4.9,84,0,0,...,0,1,0,1,0,0,1,1,0,20
2,3,"[Water, Butylene Glycol, Sucrose, Arginine, Di...","[0, 2, 5, 9, 15, 16, 23, 49, 62, 63, 64, 65, 6...",Essential Power Skin Refiner_Moisture,Laneige,28.20,5.0,12,0,0,...,0,1,1,0,0,1,1,1,0,38


In [123]:
products_and_ingredients['ing#List'] = products_and_ingredients['ing#List'].apply(lambda x: str(x))
products_and_ingredients['ing#List'] = products_and_ingredients['ing#List'].str.replace(',', '')
products_and_ingredients['ing#List'] = products_and_ingredients['ing#List'].str.replace('[', '')
products_and_ingredients['ing#List'] = products_and_ingredients['ing#List'].str.replace(']', '')
products_and_ingredients = products_and_ingredients.drop_duplicates(subset = 'product')

products_and_ingredients.head()
len(products_and_ingredients)

15950

# Recommendation engine
Based on the one detailed [here](https://www.datacamp.com/community/tutorials/recommender-systems-python)

In [131]:
#Import TfIdfVectorizer from scikit-learn
#Convert a collection of raw documents to a matrix of TF-IDF features.
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object
tfidf = TfidfVectorizer()
tfidf = TfidfVectorizer(stop_words=['Water'])

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(products_and_ingredients['ing#List'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(15950, 6890)

In [132]:
tfidf_matrix

<15950x6890 sparse matrix of type '<class 'numpy.float64'>'
	with 421810 stored elements in Compressed Sparse Row format>

In [133]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [134]:
indices = pd.Series(products_and_ingredients.index, index=products_and_ingredients['product']).drop_duplicates()

In [135]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(product, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[product]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    product_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return products_and_ingredients[['product', 'ingList']].iloc[product_indices]

In [140]:
get_recommendations('Foot Cream Norwegian Formula')

,product,ingList
5468,Daily Serum,"[Water, Butylene Glycol, Glycerin, Keratin Ami..."
5799,Environmental Rescue Mask,"[Water, Butylene Glycol, Glycerin, Keratin Ami..."
7909,Gesichtswasser Soft & Clear,"[Water, Sodium Chloride, Glycerin, Sodium Hydr..."
14697,Private Formula Day Cream Moisturiser,"[Water, Sesamum Indicum (Sesame) Seed Oil, Dis..."
9567,Ultra-Gentle Facial Moisturizer (Oil Free),"[Water, Methylparaben, Ethylparaben, Propylpar..."
1165,Moisture Oil-Free for Sensitive Skin (Canadian...,"[Water, Methylparaben, Ethylparaben, Propylpar..."
1171,"Oil-Free Moisture, for Sensitive Skin","[Water, Methylparaben, Ethylparaben, Propylpar..."
7485,Oil-Free Moisturizing Facial Lotion,"[Water, Methylparaben, Ethylparaben, Propylpar..."
5020,Dual Treatment Moisture Lotion with SPF 15,"[Water, Methylparaben, Ethylparaben, Propylpar..."
1252,Oil-Free Moisture Sensitive Skin Ultra-Gentle ...,"[Water, Methylparaben, Ethylparaben, Propylpar..."


In [144]:
ingTest = products_and_ingredients[products_and_ingredients['product'].str.match('Foot Cream Norwegian Formula')]
ingTest = ingTest['ingList']
print(*ingTest, sep='\n')

['Water', 'Sodium Chloride', 'Methylparaben', 'Propylparaben', 'Glycerin', 'Cetearyl Alcohol', 'Petrolatum', 'Cyclomethicone', 'Panthenol', 'Dimethicone', 'Keratin Amino Acids', 'Menthol', 'Allantoin', 'Dilauryl Thiodipropionate', 'Alpha-Bisabolol', 'Tocopheryl Linoleate', 'Sodium Cetearyl Sulfate', 'Sodium Sulfate', 'Diazolidinyl Urea', 'Fragrance']


'\n'